In [1]:
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from scipy.signal import find_peaks
from scipy import signal
from scipy.integrate import trapz
from scipy.interpolate import interp1d
import seaborn as sns
import heartpy as hp
import os
import pickle
import warnings
warnings.simplefilter('ignore', np.RankWarning)

In [2]:
def plt_performance(history, path='./', filename=''):
    epoch = list(range(0,len(history.history['loss'])))
    
    fig, (his_accuracy, his_loss) = plt.subplots(nrows=2,ncols=1,figsize=(10,8),sharex=True)
    his_loss.plot(epoch, history.history['loss'], label='Training loss')
    his_loss.plot(epoch, history.history['val_loss'], label='Validation loss')
    his_loss.set_xlabel("Epochs", fontsize=14)
    his_loss.set_ylabel("Loss", fontsize=14)
    his_loss.set_title("Loss", fontsize=14)
    his_loss.legend(loc='best')
    
    his_accuracy.plot(epoch, history.history['accuracy'], label='Training accuracy')
    his_accuracy.plot(epoch, history.history['val_accuracy'], label='Validation accuracy')
    his_accuracy.set_xlabel("Epochs", fontsize=14)
    his_accuracy.set_ylabel("Accuracy", fontsize=14)
    his_accuracy.set_title("Accuracy", fontsize=14)
    his_accuracy.legend(loc='best')
    
    fig.tight_layout()
    if filename == '':
        plt.savefig(f'{path}/performance.png', bbox_inches='tight')
    else:
        plt.savefig(f'{path}/{filename}.png', bbox_inches='tight')
        
def preprocessing(PPG_data):
    index = 0
    x = list(range(0,len(PPG_data[0])))
    for PPG in PPG_data:
        poly = np.polyfit(x, PPG, deg=50)
        polied = np.polyval(poly, x)
        detrended = PPG - polied
        PPG_data[index] = detrended
        index += 1
        
        
    return PPG_data
    
def get_peaks(ppg_processed):
    diff_sig_2 = np.diff(ppg_processed) ## do
    peaks_2, _ = find_peaks(diff_sig_2, distance=100*(30/60), prominence=0.01)
    result = []
    peaks = []
    for i in range(len(peaks_2)):
        if diff_sig_2[peaks_2][i] > 40:
            result.append(peaks_2[i])
            peaks.append(diff_sig_2[peaks_2][i])
            
    return result, peaks, diff_sig_2

In [3]:
dir_list_ = os.listdir("./data")
dir_list_.sort()
count = 0
dir_list = []
for i in range(22):
    dir_list.append(dir_list_[i])
    
#### pop unavailable data
dir_list.pop(5)

    
print(dir_list)
df_PPG = pd.DataFrame(columns=['participant','video_number','ppg','valence','arousal'])
Valence_data = []
Arousal_data = []
PPG_data = []

file_num = 0
for file in dir_list:
    dat_file = f'./data/{file}'
    with open(dat_file, 'rb') as f:
        Channel_data = pickle.load(f,encoding='latin1')
    data = Channel_data["data"]
    labels = Channel_data["labels"]
    for video_num in range(40):
        dataP = data[video_num,38]
        dataP = dataP[384:]
        
        label = labels[video_num]
        if label[0] < 5:
            Valence = 0
        if label[0] >= 5:
            Valence = 1
        if label[1] < 5:
            Arousal = 0
        if label[1] >= 5:
            Arousal = 1
        instance = [(file_num, video_num, dataP, Valence, Arousal)]
        new_instance = pd.DataFrame(instance, columns=['participant','video_number','ppg','valence','arousal'])
        df_PPG = pd.concat([df_PPG, new_instance], ignore_index = True)
        PPG_data.append(dataP)
        Valence_data.append(Valence)
        Arousal_data.append(Arousal)
    file_num += 1
    
PPG_shape = np.array(PPG_data)
PPG_shape.shape

['s01.dat', 's02.dat', 's03.dat', 's04.dat', 's05.dat', 's07.dat', 's08.dat', 's09.dat', 's10.dat', 's11.dat', 's12.dat', 's13.dat', 's14.dat', 's15.dat', 's16.dat', 's17.dat', 's18.dat', 's19.dat', 's20.dat', 's21.dat', 's22.dat']


(840, 7680)

In [4]:
PPG_data = df_PPG['ppg']

preprocessed = preprocessing(PPG_data)

/home/sselab/anaconda3/envs/EmotionRecognition/lib/python3.8/site-packages/numpy/lib/polynomial.py:627: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/home/sselab/anaconda3/envs/EmotionRecognition/lib/python3.8/site-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [5]:
# Concatenate PPG data of each person to use min-max normalization method 
i = 1
new_PPG = []
tmp_PPG = np.empty(0)
for PPG in preprocessed:
    if i % 40 != 0:
        tmp_PPG = np.concatenate([tmp_PPG, PPG])
    else:
        tmp_PPG = np.concatenate([tmp_PPG, PPG])
        new_PPG.append(tmp_PPG)
        tmp_PPG = np.empty(0)
    i += 1
    
    
# Normalize personal PPG data and Split into per video
new_PPG = np.array(new_PPG)
new_PPG.shape[0]
new_PPG = new_PPG.reshape(new_PPG.shape[0],new_PPG.shape[1],1)

normalized = []

for PPG in new_PPG:
    # Normalized personal PPG
    scaler = MinMaxScaler()
    norm = scaler.fit_transform(PPG) * 1000
    
    # Split into per video
    for i in range(40):
        norm_video = norm[i*7680 : i*7680+7680]
        normalized.append(norm_video)

normalized = np.array(normalized)
normalized.shape

(840, 7680, 1)

In [6]:
import neurokit2.ppg.ppg_findpeaks as findpeaks

In [7]:
segmented = []
index = 0
cal= 0

index = 0
for PPG in normalized:
    
    tmp = []
    result = findpeaks(preprocessed[index], sampling_rate=128)
    a = result['PPG_Peaks']
    
    for idx in a:
        a = idx
        if a > 70:
            seg = PPG[a-70:a+70]
            times = list(range(0,len(seg)))
        else:
            seg = PPG[0:a+70]
            times = list(range(0,len(seg)))
        if len(seg) == 140:
            cal+=1
            tmp.append(seg)

    segmented.append(tmp)
    index += 1

In [8]:
# for seg in segmented:
#     if len(seg) > 55:
#         print(len(seg))
print(len(segmented))
valence_data = []
arousal_data = []

index = 0
seger = []
for seg in segmented:
    for i in seg:
        valence_data.append(Valence_data[index])
        arousal_data.append(Arousal_data[index])
        seger.append(i)
    index += 1

840


In [9]:
vid_num = 0
for seg in segmented:
    plt.figure(figsize=(70,35))
    times = list(range(0,140))
    
    for PPG in seg:
        plt.plot(times,PPG)
    
    plt.savefig(f'segmented_data1seconds/{int(vid_num/40)+1}_{vid_num%40}.png', bbox_inches='tight')
    plt.close()
    vid_num += 1